# Fall Pervention - Data Analysis Document Review

### Table of Content:
1. [Background](#1)
2. [Creating a Dataset](#2)
3. [Choosing a Model](#3)
> 3.a. [Decision Tree](#3.a) <br>
> 3.b. [Transformer Neural Network](#3.b) <br>
> 3.c. [Recurrent Neural Network](#3.c)
> 3.d. [Conclusion](#3.d)




<a name="1"></a> 
### 1. Background:

TODO

<a name="2"></a> 
### 2. Creating a Dataset:

TODO

<a name="3"></a> 
### 3. Choosing a Model:

In order to predict the patient's current status (back lying, side lying, getting up), we should create a prediction model (Using Machine Learning or Deep Learning) which will use our dataset.<br>
Best suited models to our prediction problem:

<a name="3.a"></a> 
### 3.a. Decision Tree:
Background <br>
####Pros:
- Explainable - ...
- Fast decision making - ...

####Cons:
- Work on only 1 sample to predict - ...
- Heavy on space - ...

<a name="3.b"></a> 
### 3.b. Transformer Neural Network:
Background <br>
####Pros:
- High expexted prediction rate - ...
- Take into account the relations between the features- ...

####Cons:
- Work on only 1 sample to predict - ...
- Zero to no explainability - ...
- Harder to train - ...

<a name="3.c"></a> 
### 3.c.  Recurrent Neural Network:
Background <br>
####Pros:
- High expexted prediction rate - ...
- Works on data streams - ...

####Cons:
- Zero to no explainability - ...
- Harder to train - ...

<a name="3.d"></a> 
### 3.d.  Conclusion:
